# **Setting up Libraries**

In [ ]:
pip install gtts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install deepspeech

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 14.0 MB/s eta 0:00:00


In [ ]:
!pip install ds_ctcdecoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.9 MB/s eta 0:00:00


In [ ]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.pbmm
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.scorer

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  180M  100  180M    0     0   170M      0  0:00:01  0:00:01 --:--:--  225M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  909M  100  909M    0     0   190M      0  0:00:04  0:00:04 --:--:--  202M


In [ ]:
import tensorflow as tf
import numpy as np
import scipy.io.wavfile as wav
import math
from IPython.display import Audio, display
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import soundfile
import random
from gtts import gTTS
import deepspeech
import wave
import tensorflow.compat.v1 as tfv1
from ds_ctcdecoder import Alphabet, UTF8Alphabet,ctc_beam_search_decoder, Scorer
import scipy.io.wavfile as wav
from tensorflow.python.keras.backend import ctc_label_dense_to_sparse

In [ ]:
#@title ## Enter Original Audio File

#@markdown ---
#@markdown ### Enter a file path:
input = "normal0.wav" #@param {type:"string"}
#@markdown ---



In [ ]:
import wave
model = deepspeech.Model('deepspeech-0.9.3-models.pbmm')
audio_file = 'normal0.wav'
scorer_file_path = 'deepspeech-0.9.3-models.scorer'

# Load DeepSpeech model and scorer

scorer = model.enableExternalScorer(scorer_file_path)
# Define function to transcribe audio using the DeepSpeech model and scorer
def transcribe_audio(audio_file_path, model):
    # Read audio file
    with wave.open(audio_file_path, 'rb') as audio_file:
        audio = audio_file.readframes(audio_file.getnframes())
        audio = np.frombuffer(audio, dtype=np.int16)

    # Transcribe audio
    transcription = model.stt(audio)
    return transcription

FEATURE EXTRACTION

In [ ]:
_MEL_BREAK_FREQUENCY_HERTZ = 700.0
_MEL_HIGH_FREQUENCY_Q = 1127.0
def freq2mel(frequencies_hertz):
    """Convert frequency to mel frequency"""
    return _MEL_HIGH_FREQUENCY_Q * np.log1p(frequencies_hertz / _MEL_BREAK_FREQUENCY_HERTZ)

In [ ]:
def compute_mfcc_mel_filterbank(spectrogram, sample_rate=16000, lower_edge_hertz=20.0,upper_edge_hertz=8000.0, filterbank_channel_count=40, dtype=tf.float32, name=None):
  input_length = spectrogram.shape[-1] #is 257
  batch_size = spectrogram.shape[0] #is 1
  center_freqs = np.zeros((filterbank_channel_count + 1,))
  mel_low = freq2mel(lower_edge_hertz)
  mel_high = freq2mel(upper_edge_hertz)
  mel_span = mel_high - mel_low
  mel_spacing = mel_span / (filterbank_channel_count + 1)
  for i in range(filterbank_channel_count + 1):
      center_freqs[i] = mel_low + (mel_spacing * (i + 1))

  hz_per_sbin = 0.5 * sample_rate / (input_length - 1)
  start_index = int(1.5 + (lower_edge_hertz / hz_per_sbin))
  end_index = int(upper_edge_hertz / hz_per_sbin)

  band_mapper = np.zeros((input_length,))
  channel = 0
  for i in range(input_length):
      melf = freq2mel(i * hz_per_sbin)
      if (i < start_index) or (i > end_index):
          band_mapper[i] = -2
      else:
          while (channel < filterbank_channel_count) and (center_freqs[channel] < melf):
              channel += 1
          band_mapper[i] = channel - 1
  weights = np.zeros((input_length,))
  for i in range(input_length):
    channel = band_mapper[i]
    if (i < start_index) or (i > end_index):
        weights[i] = 0.0
    else:
        channel = int(channel)
        if (channel >= 0):
            weights[i] = (center_freqs[channel + 1] - freq2mel(i * hz_per_sbin)) / (center_freqs[channel + 1] - center_freqs[channel])
        else:
            weights[i] = (center_freqs[0] - freq2mel(i *hz_per_sbin)) / (center_freqs[0] - mel_low)
  bad_channels = []
  for c in range(filterbank_channel_count):
      band_weight_sum = 0.0
      for i in range(input_length):
          if band_mapper[i] == (c - 1):
              band_weight_sum += (1.0 - weights[i])
          elif (band_mapper[i] == c):
              band_weight_sum += weights[i]
      if (band_weight_sum < 0.5):
          bad_channels.append(c)
  mapping_matrix_w = np.zeros((filterbank_channel_count, input_length))
  mapping_matrix_r = np.zeros((filterbank_channel_count, input_length))

  for i in range(start_index, end_index):
    channel = int(band_mapper[i])
    if (channel >= 0):
      mapping_matrix_w[channel][i] = 1.0
    channel += 1
    if (channel < filterbank_channel_count):
      mapping_matrix_r[channel][i] = 1.0

  mapping_matrix_w = np.stack([mapping_matrix_w] * batch_size, axis=0)
  mapping_matrix_r = np.stack([mapping_matrix_r] * batch_size, axis=0)
  weights = np.stack([np.stack([weights] * batch_size, axis=0)] * spectrogram.shape[1], axis=1)

  spec_val = tf.math.sqrt(spectrogram)
  # spec_val_expanded = tf.expand_dims(spec_val, -1)  # expand last dimension
  weighted = tf.math.multiply(spec_val, weights)
  res = spec_val - weighted

  weighted = tf.transpose(weighted, perm=[0, 2, 1])
  res = tf.transpose(res, perm=[0, 2, 1])

  vals = tf.matmul(tf.cast(mapping_matrix_w, tf.float32), weighted) + tf.matmul(tf.cast(mapping_matrix_r, tf.float32), res)
  return vals

In [ ]:
def create_dct_matrix(dct_coefficient_count=13, filterbank_channel_count=40, dtype=tf.float32):
    fnorm = np.sqrt(2 / filterbank_channel_count)
    arg = np.pi / filterbank_channel_count

    arg_ = np.zeros((dct_coefficient_count, filterbank_channel_count))
    for i in range(dct_coefficient_count):
        for j in range(filterbank_channel_count):
            arg_[i][j] = i * arg * (j + 0.5)

    return fnorm * np.cos(arg_)

In [ ]:
def compute_dct(log_mel_energies, dct_coefficient_count=26, filterbank_channel_count=40, dtype=tf.float32):
  batch_size = log_mel_energies.shape[0]
  cosines = create_dct_matrix(dct_coefficient_count, filterbank_channel_count)

  # Make cosines to have shape (batch_size, dct_coefficient_count, filterbank_channel_count)
  cosines = tf.cast(np.stack([cosines] * batch_size, axis=0), dtype)

  input_length = log_mel_energies.shape[1]
  if input_length > filterbank_channel_count:
      input_length = filterbank_channel_count

  # return a Tensor of shape (batch_size, N, dct_coefficient_count)
  return tf.transpose(tf.matmul(cosines[:, :input_length, :], log_mel_energies), perm=[0, 2, 1])

NEURAL NETWORK AND LOGITS


In [ ]:
# Value assigning
n_cell_dim = 2048
cpu_device = '/cpu:0'
alphabet = UTF8Alphabet()
n_hidden = n_hidden_1 = n_hidden_2 = n_hidden_3 = 2048
n_hidden_6 = alphabet.GetSize() + 1
lower_edge_hertz=20.0
upper_edge_hertz=8000.0
sample_rate=16000
n_input = 26

In [ ]:
def compute_mfcc(spectrogram, sample_rate, lower_edge_hertz=20,upper_edge_hertz=4000, filterbank_channel_count=40, dct_coefficient_count=13, dtype=tf.float32):
    # Compute mfcc filterbanks.
    vals = compute_mfcc_mel_filterbank(spectrogram, sample_rate, lower_edge_hertz, upper_edge_hertz, filterbank_channel_count)

    # Set small values to 1e-12 so that log calculation doesn't run into trouble.
    kFilterbankFloor = 1e-12
    vals = vals * tf.cast(vals > 0, dtype) + kFilterbankFloor
    log_mel_energies = tf.math.log(vals)

    # Return results of DCT.
    return compute_dct(log_mel_energies, dct_coefficient_count)

In [ ]:
def variable_on_cpu(name, shape, initializer):
    # Use the /cpu:0 device for scoped operations
    with tf.device(cpu_device):
        # Create or get apropos variable
        var = tfv1.get_variable(name=name, shape=shape, initializer=initializer)
    return var

In [ ]:
def dense(name, x, units, dropout_rate=None, relu=True, layer_norm=False):
    with tfv1.variable_scope(name):
        bias = variable_on_cpu('bias', [units], tfv1.zeros_initializer())
        weights = variable_on_cpu('weights', [x.shape[-1], units], tfv1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"))

    output = tf.nn.bias_add(tf.matmul(x, weights), bias)

    if relu:
        output = tf.minimum(tf.nn.relu(output), 20.0)

    if layer_norm:
        with tfv1.variable_scope(name):
            output = tf.contrib.layers.layer_norm(output)

    if dropout_rate is not None:
        output = tf.nn.dropout(output, rate=dropout_rate)

    return output

In [ ]:
def rnn_impl_lstmblockfusedcell(x, seq_length, previous_state, reuse):
    with tfv1.variable_scope('cudnn_lstm/rnn/multi_rnn_cell/cell_0'):
        fw_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(n_cell_dim,
                                                    forget_bias=0,
                                                    reuse=reuse,
                                                    name='cudnn_compatible_lstm_cell')

        output, output_state = fw_cell(inputs=x,
                                       state=previous_state)

    return output, output_state

In [ ]:
def create_overlapping_windows(batch_x):
  n_context = 9
  n_input = 26
  batch_size = tf.shape(input=batch_x)[0]
  window_width = 2 * n_context + 1
  num_channels = n_input

  # Create a constant convolution filter using an identity matrix, so that the
  # convolution returns patches of the input tensor as is, and we can create
  # overlapping windows over the MFCCs.
  eye_filter = tf.constant(np.eye(window_width * num_channels).reshape(window_width, num_channels, window_width * num_channels), tf.float32) # pylint: disable=bad-continuation

  # Create overlapping windows
  batch_x = tf.nn.conv1d(input=batch_x, filters=eye_filter, stride=1, padding='SAME')

  # Remove dummy depth dimension and reshape into [batch_size, n_windows, window_width, n_input]
  batch_x = tf.reshape(batch_x, [batch_size, -1, window_width, num_channels])

  return batch_x

In [ ]:
def create_model(batch_x, seq_length, dropout, reuse=False, batch_size=None, previous_state=(tf.zeros([159, n_cell_dim]), tf.zeros([159, n_cell_dim])), overlap=True):
  layers = {}
  batch_size = tf.shape(input=batch_x)[0]
  batch_x = tf.transpose(a=batch_x, perm=[1 ,0, 2,3])
  n_context = 9
  n_input = 26

  # Reshape to prepare input for first layer
  batch_x = tf.reshape(batch_x, [-1, n_input + 2*n_input*n_context])
  layers['input_reshaped'] = batch_x

  n_hidden = n_hidden_1 = n_hidden_2 = n_hidden_3 = 2048
  # The next three blocks will pass `batch_x` through three hidden layers with
  # clipped RELU activation and dropout.
  layers['layer_1'] = layer_1 = dense('layer_1', batch_x, n_hidden_1, dropout_rate=dropout[0], layer_norm=False)
  layers['layer_2'] = layer_2 = dense('layer_2', layer_1, n_hidden_2, dropout_rate=dropout[1], layer_norm=False)
  layers['layer_3'] = layer_3 = dense('layer_3', layer_2, n_hidden_3, dropout_rate=dropout[2], layer_norm=False)

  # `layer_3` is now reshaped into `[n_steps, batch_size, 2*n_cell_dim]`,
  # as the LSTM RNN expects its input to be of shape `[max_time, batch_size, input_size]`.
  # layer_3 = tf.reshape(layer_3, [-1, batch_size, n_hidden_3])
  # Run through parametrized RNN implementation, as we use different RNNs
  # for training and inference
  output, output_state = rnn_impl_lstmblockfusedcell(layer_3, seq_length, previous_state , reuse )

  # Reshape output from a tensor of shape [n_steps, batch_size, n_cell_dim]
  # to a tensor of shape [n_steps*batch_size, n_cell_dim]
  output = tf.reshape(output, [-1,2048])
  layers['rnn_output'] = output
  layers['rnn_output_state'] = output_state

  # Now we feed `output` to the fifth hidden layer with clipped RELU activation
  layers['layer_5'] = layer_5 = dense('layer_5', output, 2048, dropout_rate=dropout[5], layer_norm=False)

  # Now we apply a final linear layer creating `n_classes` dimensional vectors, the logits.
  layers['layer_6'] = layer_6 = dense('layer_6', layer_5, n_hidden_6, relu=False)

  # Finally we reshape layer_6 from a tensor of shape [n_steps*batch_size, n_hidden_6]
  # to the slightly more useful shape [n_steps, batch_size, n_hidden_6].
  # Note, that this differs from the input in that it is time-major.
  layer_6 = tf.reshape(layer_6, [-1, batch_size, n_hidden_6], name='raw_logits')
  layers['raw_logits'] = layer_6
  return layer_6

In [ ]:
def periodic_hann_window(window_length, dtype):
    return 0.5 - 0.5 * tf.math.cos(2.0 * np.pi * tf.range(tf.cast(window_length,tf.float32), dtype=dtype) / (tf.cast(window_length,tf.float32)))

def get_logits(audio, length):
  audio = tf.cast(audio / 2 ** 15, tf.float32)
  # print(audio.shape)
  # performs short-time Fourier transform (STFT) on the audio signal 
  stfts = tf.signal.stft(
          audio,
          frame_length=512,
          frame_step=320,
          fft_length=512,
          window_fn=periodic_hann_window)

  # power spectral density of the audio signal at each time-frequency point in the STFT
  spectrogram = tf.square(tf.abs(stfts))
  features = compute_mfcc(spectrogram=spectrogram, sample_rate=sample_rate,
                                 upper_edge_hertz=sample_rate / 2, dct_coefficient_count=n_input)
  features = create_overlapping_windows(features)
  # print(features)
  no_dropout = [None] * 6
  logits = create_model(features, seq_length=length,dropout=no_dropout, overlap=False)
  return logits

In [ ]:
input = 'normal0.wav'
fs, audioip = wav.read(input)
assert audioip.dtype == np.int16

audios = []
lengths = []
source_dB = 20 * np.log10(np.max(np.abs(audioip)))
length = len(audioip)
audios.append(list(audioip))
lengths.append(len(audioip))

print("Start attack..\n")
batch_size = 1
phrase_length = len('without the data the articles useless')
max_audio_len = len(audioip)
learning_rate = 10
audio_sample_rate = 16000
feature_win_step = 20
audio_step_samples = audio_sample_rate * (feature_win_step / 1000)
phrase = 'the quick brown fox jumped over the lazy dog'
toks = " abcdefghijklmnopqrstuvwxyz'-"
target = [[toks.index(x) for x in phrase]]*len(audios)

#Initializing everything
delta = tf.Variable(np.zeros((batch_size, max_audio_len), dtype=np.float32), name='qq_delta')
mask = np.array([[1 if i < l else 0 for i in range(max_audio_len)] for l in lengths])
original = np.array(audioip)
target_phrase = np.array([list(t)+[0]*(phrase_length-len(t)) for t in target])
target_phrase_lengths = np.array([len(x) for x in target])
lengths = (np.array(lengths)-(2*audio_step_samples/3))//320
rescale = tf.Variable(np.zeros((batch_size,1), dtype=np.float32), name='qq_phrase_lengths')
cwmask = np.array([[1 if i < l else 0 for i in range(phrase_length)] for l in (np.array(lengths)-1)//320])
c = np.ones((batch_size, phrase_length))
importance = c

# bound the l_infty norm by 2000
apply_delta = tf.clip_by_value(delta, -2000, 2000)*rescale

# set the new input to the model to be the above delta plus a mask
new_input = apply_delta*mask + original

# add a tiny bit of noise
noise = tf.random.normal(new_input.shape,stddev=2)
pass_in = tf.clip_by_value(new_input+noise, -2**15, 2**15-1)
audio = pass_in
audio = tf.cast(audio / 2 ** 15, tf.float32)
# performs short-time Fourier transform (STFT) on the audio signal 
stfts = tf.signal.stft(
        audio,
        frame_length=512,
        frame_step=320,
        fft_length=512,
        window_fn=periodic_hann_window)

# power spectral density of the audio signal at each time-frequency point in the STFT
spectrogram = tf.square(tf.abs(stfts))
features = compute_mfcc(spectrogram=spectrogram, sample_rate=sample_rate,
                                upper_edge_hertz=sample_rate / 2, dct_coefficient_count=n_input)
features = create_overlapping_windows(features)
no_dropout = [None] * 6
logits =  get_logits(audio,lengths)

Start attack..



<ipython-input-370-069ead2dd4be>:3: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  fw_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(n_cell_dim,


In [ ]:
print(logits)

tf.Tensor(
[[[-2.56231308e-01  2.34421790e-01 -3.02859664e-01 ... -1.77050114e-01
    2.20381215e-01 -3.36979926e-01]]

 [[-3.84476967e-02  3.47266287e-01 -1.16438285e-01 ... -1.50487244e-01
    2.83463269e-01 -4.34533805e-01]]

 [[ 1.00131206e-01  4.39821064e-01  1.99281517e-02 ... -1.93391562e-01
    2.01496840e-01 -3.85772973e-01]]

 ...

 [[-4.18566167e-04  1.51893437e-01 -3.82079929e-03 ... -1.05715752e-01
    4.16137278e-04 -1.51061118e-01]]

 [[ 4.92954366e-02  2.63503730e-01 -8.86681080e-02 ... -9.59640220e-02
   -1.22602209e-01 -4.49339300e-01]]

 [[ 2.79167444e-02  2.01020479e-01 -6.46790266e-02 ... -1.14973590e-01
   -1.03740603e-01 -5.04016876e-01]]], shape=(159, 1, 256), dtype=float32)


In [ ]:
final_deltas = [None]*batch_size
MAX = num_iterations = 5000
first_hits = np.zeros((batch_size,))
best_hits = np.zeros((batch_size,))

In [ ]:
def get_probs(delta,mask,original):
  apply_delta = tf.clip_by_value(delta, -2000, 2000)*rescale

  # set the new input to the model to be the above delta plus a mask
  new_input = apply_delta*mask + original

  # add a tiny bit of noise
  noise = tf.random.normal(new_input.shape,stddev=2)
  pass_in = tf.clip_by_value(new_input+noise, -2**15, 2**15-1)
  pass_in = tf.cast(pass_in / 2 ** 15, tf.float32)
  stfts = tf.signal.stft(
        pass_in,
        frame_length=512,
        frame_step=320,
        fft_length=512,
        window_fn=periodic_hann_window)
  spectrogram = tf.square(tf.abs(stfts))
  features = compute_mfcc(spectrogram=spectrogram, sample_rate=sample_rate,
                                  upper_edge_hertz=sample_rate / 2, dct_coefficient_count=n_input)
  features = create_overlapping_windows(features)
  no_dropout = [None] * 6
  logits =  get_logits(audio,lengths)
  probs = tf.squeeze(tf.nn.softmax(logits, name='logits'))
  return probs

In [ ]:
probs = get_probs(delta,mask,original)

<ipython-input-370-069ead2dd4be>:3: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  fw_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(n_cell_dim,


In [ ]:
from ds_ctcdecoder import ctc_beam_search_decoder, Scorer
lm_alpha = 0.931289039105002
lm_beta = 1.1834137581510284
scorer_path = '/content/deepspeech-0.9.3-models.scorer'
alphabet = UTF8Alphabet()
scorer = Scorer(lm_alpha, lm_beta,scorer_path, alphabet)

In [ ]:
l2penalty = float('inf')

In [ ]:
target_phrase = tf.cast(target_phrase, tf.int32)
target_phrase_lengths = tf.cast(target_phrase_lengths, tf.int32)

In [ ]:
from tensorflow.python.keras.backend import ctc_label_dense_to_sparse
# @tf.function
# def loss_func(target_phrase,target_phrase_lengths,logits,new_input,original,l2penalty):
def ctcloss_func(target_phrase,target_phrase_lengths,logits): 
  target_phrase = tf.cast(target_phrase, tf.int32)
  target_phrase_lengths = tf.cast(target_phrase_lengths, tf.int32)
  target = ctc_label_dense_to_sparse(target_phrase, target_phrase_lengths)
  ctcloss = tf.compat.v1.nn.ctc_loss(labels=tf.cast(target, tf.int32),inputs=logits, sequence_length=target_phrase_lengths)
  return ctcloss

def loss_func(new_input,original,ctcloss):
  if not np.isinf(l2penalty):
    loss = tf.reduce_mean((new_input-original)**2,axis=1) + l2penalty*ctcloss
    print('im here')
  else:
    loss = ctcloss
  return loss

def expanded_loss_func():
  expanded_loss = tf.constant(0)  

In [ ]:
ctcloss = ctcloss_func(target_phrase,target_phrase_lengths,logits)
loss = loss_func(new_input,original,ctcloss)

In [ ]:
print(ctcloss)

tf.Tensor([248.46167], shape=(1,), dtype=float32)


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
with tf.GradientTape() as tape:
  tape.watch(logits)
  target = ctc_label_dense_to_sparse(target_phrase, target_phrase_lengths)
  ctcloss = tf.compat.v1.nn.ctc_loss(labels=tf.cast(target, tf.int32),inputs=logits, sequence_length=target_phrase_lengths)
  loss = ctcloss
grad,var = zip(optimizer.compute_gradients(loss, [logits],tape=tape)[0])
# grad_var = zip(optimizer.compute_gradients(loss, [logits],tape=tape)[0])
train = optimizer.apply_gradients(grad_var)

In [ ]:
print(grad)

(<tf.Tensor: shape=(159, 1, 256), dtype=float32, numpy=
array([[[0.00295967, 0.00483427, 0.00282483, ..., 0.00320355,
         0.00476687, 0.00273007]],

       [[0.00367779, 0.00540879, 0.00340185, ..., 0.00328797,
         0.00507447, 0.00247496]],

       [[0.00423182, 0.00594364, 0.00390567, ..., 0.00315538,
         0.00468328, 0.00260317]],

       ...,

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]]], dtype=float32)>,)


In [ ]:
loss_fn = 'CTC'

In [ ]:
target = [[toks.index(x) for x in phrase]]*len(audios)
print(type(target))

<class 'list'>


In [ ]:
for i in range(2):
  if i%10 == 0:
    new, delta, probs_out, r_logits = new_input, delta, get_probs(delta,mask,original), logits
    lst = [(probs_out, r_logits)]
    batch_size = r_logits.shape[1]
    for out, logits in lst:
        out_list = []
        for ii in range(batch_size):
            if batch_size == 1:
                probs = probs_out
            else:
                probs = probs_out[:,ii,:]
            decoded = tf.nn.ctc_beam_search_decoder(logits, lengths, beam_width=1000)
            decoded_sparse = decoded[0]
            decoded_dense = tf.sparse.to_dense(decoded_sparse[0])
            decoded_sequences = tf.strings.reduce_join(tf.strings.as_string(decoded_dense + 1), axis=1)
            out_list.append(decoded_sequences)
  feed_dict = {}
  target_phrase = tf.cast(target_phrase, tf.int32)
  target_phrase_lengths = tf.cast(target_phrase_lengths, tf.int32)
  d, el, cl, l, logits, new_input, _ = delta, expanded_loss_func(), ctcloss_func(target_phrase,target_phrase_lengths,logits), \
                                      loss_func(new_input,original,ctcloss), get_logits(pass_in,lengths), new_input,train
  print("%.3f"%np.mean(cl), "\t", "\t".join("%.3f"%x for x in cl))
  logits = np.argmax(logits,axis=2).T
  for ii in range(batch_size):
      # Every 100 iterations, check if we've succeeded
      # if we have (or if it's the final epoch) then we
      # should record our progress and decrease the
      # rescale constant.
      if (loss_fn == "CTC" and i%10 == 0 and out_list[ii] == "".join([toks[x] for x in target[ii]])) \
          or (i == MAX-1 and final_deltas[ii] is None):
          # Get the current constant
          # print('im in loss fn for loop')
          rescale = rescale
          if rescale[ii]*2000 > np.max(np.abs(d)):
              # If we're already below the threshold, then
              # just reduce the threshold to the current
              # point and save some time.
              print("It's way over", np.max(np.abs(d[ii]))/2000.0)
              rescale[ii] = np.max(np.abs(d[ii]))/2000.0

          # Otherwise reduce it by some constant. The closer
          # this number is to 1, the better quality the result
          # will be. The smaller, the quicker we'll converge
          # on a result but it will be lower quality.
          rescale[ii].assign(rescale[ii] * .8)

          # Adjust the best solution found so far
          # final_deltas[ii] = new_input[ii]

          print("Worked i=%d ctcloss=%f bound=%f"%(ii, cl[ii], 2000*rescale[ii][0]))
          if (first_hits[ii] == 0):
              print("First hit for audio {} at iteration {}".format(ii, i))
              first_hits[ii]=i
          else:
              best_hits[ii]=i

          rescale=rescale

          print('im saving audio file okay')
          wav.write("tmp/adv.wav", 16000, np.array(np.clip(np.round(new_input[ii]), -2**15, 2**15-1),dtype=np.int16))

<ipython-input-370-069ead2dd4be>:3: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  fw_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(n_cell_dim,


248.462 	 248.462


InvalidArgumentError: ignored

In [ ]:
import IPython.display as ipd
ipd.Audio('normal0.wav')

In [ ]:
import IPython.display as ipd
ipd.Audio('tmp/adv.wav')

ValueError: ignored